In [3]:
import pandas_datareader as pdr
from datetime import datetime
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

### Download data & run model

*Note*
- The program is set to automatically download stock prices 2018/2/1 ~ 2019/5/9 for prediction on 5/10
- Runtime ~30s

In [7]:
companies = ['INTC', 'AMD', 'CSCO', 'AAPL', 'MU', 'NVDA', 'QCOM', 'AMZN', 'NFLX', 'FB', 'GOOG', 'BABA', 'EBAY', 'IBM', 'XLNX', 'TXN', 'NOK', 'TSLA', 'MSFT', 'SNPS']
comp_data = {}
for ticker in companies:
    tmp = pdr.get_data_yahoo(symbols=ticker, start=datetime(2018, 2, 1), end=datetime(2019, 5, 9))
    comp_data[ticker] = tmp

# download oil price
oil = pdr.get_data_yahoo(symbols='OIL', start=datetime(2018, 2, 1), end=datetime(2019, 5, 9))

def pred_stock(pred_close_price, today_close_price):
    
    percent_change = abs(pred_close_price - today_close_price) * 100/ today_close_price
    
    if percent_change > 2.0:
        if (pred_close_price > today_close_price):
            return 0 # rise
        elif (pred_close_price <= today_close_price):
            return 2 # fall
    elif percent_change < 2.0:
        return 1 # stable
    
final_res = []
print("Result Prediction on 5/10:\n")
for cpn in companies:
    
    intl = comp_data[cpn]
    intl['oil_high'] = oil['High']
    intl['oil_low'] = oil['Low']
    intl['oil_open'] = oil['Open']
    intl['oil_close'] = oil['Close']
    
    y_train = intl.Close[1:]
    x_train = intl.iloc[:-1,:]
    #print("COMPANY\n", intl.tail())
    #print("y-TRAI\n", y_train.tail())
    #print("X-TRAIN\n", x_train.tail())
    y_train = y_train.values
    x_train = x_train.values
    
    x_test = intl.iloc[-1, :]
    #print("X-test\n", x_test)
    x_test = x_test.values.reshape(1, -1)
    #print(x_test)
    #df_x_test = MinMaxScaler().fit_transform(x_test.astype('float32'))
    #df_x_test = pd.DataFrame(df_x_test)
    
    #x_train = autoencoder.reducedimension(x_train, 5, 0.001, 128, 100)
    #x_test = autoencoder.reducedimension(x_test, 5, 0.001, 128, 100)
    
    #minmax = MinMaxScaler().fit(y_train.values.reshape((-1,1))) # y train
    
    #minmax = pd.DataFrame(minmax)
    #minmax = np.array(minmax)
    #minmax = pd.DataFrame(minmax.reshape((-1, 1)))
    
    #df_log = MinMaxScaler().fit_transform(x_train.astype('float32'))
    #df_log = pd.DataFrame(df_log) # x train
    
    #df_log = df_log.values
    
    #print('y_train minmax: {}'.format(len(minmax)))
    #print('df_log: {}'.format(len(df_log)))

    
    
    # xgbOost 
    params_xgd = {
        'max_depth': 7,
        'objective': 'reg:logistic',
        'learning_rate': 0.05,
        'n_estimators': 10000
        }
    
    #train_Y = df_log.values[start:, 3]

    #clf = xgb.XGBRegressor() ## **?
    
    #clf.fit(stack_predict,y_train)
    clf = xgb.XGBRegressor()
    #model = xgb.XGBRegressor()
    #model.fit(df_log,minmax)
    #print (model)

    clf.fit(x_train, y_train)
    
    stacked = clf.predict(x_test)

    #true_close = reverse_close(train_Y)
    #pred_close = reverse_close(stacked) # 
    
    #pred_ = pred_arima[-1]
    
    #pred_ = pred_close[-1] # t + 1 price
    
    pred_ = stacked
    today_ = y_train[-1] # t price
    
    wow_pred = pred_stock(pred_, today_)
    final_res.append(wow_pred)
    
    print('{}: {} ----> today: {} --- predicted_tomorrow: {}'.format(cpn, wow_pred, today_, pred_))



Result Prediction on 5/10:

INTC: 1 ----> today: 46.619998931884766 --- predicted_tomorrow: [46.838593]
AMD: 1 ----> today: 27.209999084472656 --- predicted_tomorrow: [26.95659]
CSCO: 1 ----> today: 52.91999816894531 --- predicted_tomorrow: [53.004677]
AAPL: 1 ----> today: 200.72000122070312 --- predicted_tomorrow: [202.88957]
MU: 1 ----> today: 39.27000045776367 --- predicted_tomorrow: [38.75364]
NVDA: 2 ----> today: 170.19000244140625 --- predicted_tomorrow: [165.81253]
QCOM: 1 ----> today: 83.77999877929688 --- predicted_tomorrow: [84.59456]
AMZN: 1 ----> today: 1899.8699951171875 --- predicted_tomorrow: [1905.7084]
NFLX: 1 ----> today: 362.75 --- predicted_tomorrow: [360.71155]
FB: 1 ----> today: 188.64999389648438 --- predicted_tomorrow: [188.98941]
GOOG: 1 ----> today: 1162.3800048828125 --- predicted_tomorrow: [1163.8878]
BABA: 1 ----> today: 179.0399932861328 --- predicted_tomorrow: [178.98433]
EBAY: 1 ----> today: 37.619998931884766 --- predicted_tomorrow: [37.550915]
IBM: 1 -

In [ ]:
# write output
file_name = '0510_0760816'
def output_txt(file_name, list_pred):
    
    with open(file_name, "w") as f:
        
        for pred in list_pred:

            f.write("%s\n"%(str(int(pred))))

    f.close()
    
output_txt(file_name, final_res)